In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchtext
from torchtext.data import Field, BucketIterator
# from torchtext.datasets import UDPOS
# from torchcrf import CRF

from tqdm import tqdm
import spacy
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
spacy_en = spacy.load("en_core_web_sm")

In [ ]:
def tokenizer_en(text):
    return text.split(" ")

In [ ]:
def get_text_labels(sequence_tags):
    # TOKEN TXT_SOURCE_FILE START_CHAR END_CHAR TAG TAG_ID ROOT_ID RELATION
    text = [data[0].strip() for data in sequence_tags]
    tags = [data[4].strip() for data in sequence_tags]
#     return {"text":text, "labels": tags}
    return {"text":" ".join(text), "labels": " ".join(tags)}
def parse_deft(deft_file):
    with open(deft_file, 'r') as deft:
        all_text = deft.read()
    all_sequences = []
    for lines in all_text.split("\n\n"):
        sents = []
        for token_data in lines.split("\n"):
            if len(token_data.split("\t"))==8:
                sents.append(token_data.split("\t"))
        all_sequences.append(get_text_labels(sents))
    return all_sequences
# parse_deft(train_deft_file)


In [ ]:
import os
train_deft_file = "../deft_corpus/data/deft_files/train/"
all_data = []
for files in os.listdir(train_deft_file):
    all_data.extend(parse_deft(os.path.join(train_deft_file, files)))
print(len(all_data))

In [ ]:
# Qualifier
for files in os.listdir(train_deft_file):
    with open(os.path.join(train_deft_file, files), 'r') as deft:
        all_text = deft.read()
    for lines in all_text.split("\n\n"):
        for token_data in lines.split("\n"):
            if len(token_data.split("\t"))==8:
                if "Alias-Term" in token_data.split("\t")[4]:
                    print(token_data)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(all_data, test_size=0.1)
len(train), len(test)

In [ ]:
train[0]

In [ ]:
import pandas as pd
pd.DataFrame.from_records(train).to_csv("train.csv", index=False)
pd.DataFrame.from_records(test).to_csv("val.csv", index=False)

In [ ]:
TEXT = Field(
    sequential=True, 
    tokenize=None,
    init_token="<sos>",
    eos_token="<eos>",
#     batch_first=True,
)
LABELS = Field(
    sequential=True, 
    init_token="<sos>",
    eos_token="<eos>",
#     batch_first=True,
    is_target=True
)

In [ ]:
from torchtext.data import TabularDataset

tv_datafields = [("text", TEXT), ("labels", LABELS)]
trn, vld = TabularDataset.splits(
               path="", # the root directory where the data lies
               train='train.csv',validation="val.csv",
               format='csv',
                skip_header=True,
               fields=tv_datafields)

In [ ]:
vars(trn.examples[0])

In [ ]:
TEXT.build_vocab(trn, vectors="glove.6B.100d")
LABELS.build_vocab(trn)
# PTB_LABELS.build_vocab(training_data)

len(TEXT.vocab), len(LABELS.vocab)

In [ ]:
vars(LABELS.vocab)